In [ ]:
import pandas as pd
import numpy as np
import argparse
import glob
# import wandb
import json
import joblib
import os

from sklearn.model_selection import KFold, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder, StandardScaler

CALC_PATHS = '/home/nfs/jludwiczak/af2_cc/af2_multimer/calc'

In [49]:
import sys
import os
import glob

import pandas as pd
import numpy as np

import pytest

from src.predictor import predict_oligo_state
DATA_PATH = 'tests/data/*'
# @pytest.mark.parametrize("use_pairwise", [True, False])

def test_predict_oligo_state():
    test_df = pd.DataFrame()
    test_cases = glob.glob(DATA_PATH)
    for i, test_case in enumerate(test_cases):
        df = predict_oligo_state(test_case, use_pairwise=True)
        df['test_case_index'] = i
        df = df.set_index('test_case_index')
        test_df = pd.concat([test_df, df], axis=0)
    print(test_df)
    # test_df.to_csv('tests/test_df.csv')
    # print(test_df)
    assertion_df = pd.read_csv('tests/test_df.csv').set_index('test_case_index')
    # diff = test_df.compare(assertion_df)
    # print(diff)
    assert np.isclose(test_df, assertion_df).all(), 'Test case results do not match assertion results'

test_predict_oligo_state()

Predicted oligomer state: Dimer (0) with probability           0.988 +/- 0.0013
Predicted oligomer state: Dimer (0) with probability           0.99977 +/- 0.00015
Predicted oligomer state: Dimer (0) with probability           0.99959 +/- 0.00025
Predicted oligomer state: Trimer (1) with probability           0.9919 +/- 0.00163
Predicted oligomer state: Tetramer (2) with probability           0.98122 +/- 0.00682
Predicted oligomer state: Tetramer (2) with probability           0.98602 +/- 0.00217
Predicted oligomer state: Trimer (1) with probability           0.9738 +/- 0.00465
Predicted oligomer state: Dimer (0) with probability           0.99682 +/- 0.00199
Predicted oligomer state: Tetramer (2) with probability           0.99407 +/- 0.00619
Predicted oligomer state: Tetramer (2) with probability           0.9657 +/- 0.00525
Predicted oligomer state: Trimer (1) with probability           0.99928 +/- 0.00049
                 prob_dimer  prob_dimer_std  prob_trimer  prob_trimer_std  \
t